# imports + device


In [ ]:
import numpy as np
import wgpu
from wgpu.utils import get_default_device

from src.data_init import make_points
from src.gpu_utils import read_text, create_storage_buffer, create_uniform_buffer
from src.data_init import make_grid_cloth
from src.data_init import make_grid_indices
from src.gpu_utils import create_index_buffer



# ------------------------------------------------------------
# Initialisation du GPU (WebGPU via wgpu)
# ------------------------------------------------------------

# Récupération du device GPU par défaut
# - abstraction multiplateforme (Vulkan / Metal / DX12 selon l’OS)
# - point d’entrée principal pour créer buffers, pipelines, shaders
device = get_default_device()

# File de commandes associée au device
# - permet d’envoyer des commandes au GPU
# - utilisée pour copier des données CPU → GPU
queue = device.queue

# Affichage du device pour vérification / debug
print("Device:", device)


# init data + params

In [ ]:
# ------------------------------------------------------------
# Paramètres de la simulation (Step 2A — Ressorts structuraux)
# ------------------------------------------------------------

# Taille de la grille (tissu) : W colonnes, H lignes
W, H = 16, 16
N = W * H  # nombre total de sommets

# Pas de temps de la simulation (Δt) : ici 60 Hz
dt = np.float32(1.0 / 60.0)

# Gravité (axe Y) : négative car Y vers le haut
g = np.float32(-9.81)

# Distance au repos entre voisins (ressort structural)
# Correspond à l'espacement initial de la grille
rest = np.float32(0.10)

mu = np.float32(0.3)  # coefficient de friction

# ------------------------------------------------------------
# Génération des données initiales (CPU) : GRILLE régulière
# ------------------------------------------------------------
#from src.data_init import make_grid_cloth

# positions : (N,4) float32  (x,y,z,w)  avec w=1
# velocities: (N,4) float32  (vx,vy,vz,w) avec w=0
positions_np, velocities_np = make_grid_cloth(W, H, float(rest))
indices_np = make_grid_indices(W, H)   # CPU
idx_buf = create_index_buffer(device, indices_np)  # GPU

print("\nOK indices:", indices_np.shape, indices_np.dtype)
print("\nOK index buffer GPU created, index_count =", indices_np.size)

# Debug : vérifier shapes + types + premières valeurs
print("\npositions shape:", positions_np.shape, positions_np.dtype)
print("\nvelocities shape:", velocities_np.shape, velocities_np.dtype)

print("\nAvant simulation - positions :", positions_np[:3])
print("\nAvant simulation - vitesses  :", velocities_np[:3])


# buffers GPU

In [ ]:
import struct
import wgpu

# ------------------------------------------------------------
# Buffers GPU : double buffering (lecture -> écriture)
# ------------------------------------------------------------
# Buffers d'entrée (READ dans le shader + VERTEX pour le rendu)
pos_in = device.create_buffer_with_data(
    data=positions_np,
    usage=wgpu.BufferUsage.STORAGE | wgpu.BufferUsage.VERTEX | wgpu.BufferUsage.COPY_SRC | wgpu.BufferUsage.COPY_DST
)
vel_in = device.create_buffer_with_data(
    data=velocities_np,
    usage=wgpu.BufferUsage.STORAGE | wgpu.BufferUsage.COPY_SRC | wgpu.BufferUsage.COPY_DST
)

# Buffers de sortie (WRITE dans le shader + VERTEX pour le rendu)
pos_out = device.create_buffer(
    size=positions_np.nbytes,
    usage=wgpu.BufferUsage.STORAGE | wgpu.BufferUsage.VERTEX | wgpu.BufferUsage.COPY_SRC | wgpu.BufferUsage.COPY_DST
)
vel_out = device.create_buffer(
    size=velocities_np.nbytes,
    usage=wgpu.BufferUsage.STORAGE | wgpu.BufferUsage.COPY_SRC | wgpu.BufferUsage.COPY_DST
)

# ------------------------------------------------------------
# Buffer d'indices pour le rendu (NOUVEAU)
# ------------------------------------------------------------
indices_np = make_grid_indices(W, H)  # ← Utilise ta fonction !
idx_buf = device.create_buffer_with_data(
    data=indices_np,
    usage=wgpu.BufferUsage.INDEX | wgpu.BufferUsage.COPY_DST
)
print(f"✅ Index buffer créé : {indices_np.size} indices ({indices_np.size // 3} triangles)")

# ------------------------------------------------------------
# Paramètres physiques (à ajuster si instable)
# ------------------------------------------------------------
k = np.float32(60.0)        # raideur du ressort (Hooke)
mass = np.float32(1.0)      # masse par point
damping = np.float32(0.995)  # amortissement (stabilité)
mu = np.float32(0.1)        # friction

# ------------------------------------------------------------
# Buffer UNIFORM : Params (doit matcher le struct WGSL)
# ------------------------------------------------------------
radius = np.float32(1.0)
sphere_c = (np.float32(0.0), np.float32(-2.5), np.float32(0.0))

params_bytes = struct.pack(
    "ffffffffIIIIffff",
    float(dt), float(g), float(k), float(rest),
    float(mass), float(damping), float(radius), float(mu),
    int(W), int(H), int(N), 0,
    float(sphere_c[0]), float(sphere_c[1]), float(sphere_c[2]), 0.0
)
params_buf = create_uniform_buffer(device, params_bytes)
print("✅ Buffers pos_in/vel_in + pos_out/vel_out + params + indices créés")

In [ ]:
idx_buf = create_index_buffer(device, indices_np)

print("Index buffer OK")
print("Nb indices:", indices_np.size)


# pipeline + bind groups

In [ ]:
shader_code = read_text("shaders/step4_collision_friction.wgsl")
shader_module = device.create_shader_module(code=shader_code)

# ------------------------------------------------------------
# Layout du bind group (doit matcher le WGSL Step 2A)
# ------------------------------------------------------------
# WGSL :
# @binding(0) pos_in   : storage READ
# @binding(1) vel_in   : storage READ
# @binding(2) pos_out  : storage READ/WRITE
# @binding(3) vel_out  : storage READ/WRITE
# @binding(4) params   : uniform

bind_group_layout = device.create_bind_group_layout(entries=[
    {
        "binding": 0,
        "visibility": wgpu.ShaderStage.COMPUTE,
        "buffer": {"type": "read-only-storage"}  # pos_in (lecture)
    },
    {
        "binding": 1,
        "visibility": wgpu.ShaderStage.COMPUTE,
        "buffer": {"type": "read-only-storage"}  # vel_in (lecture)
    },
    {
        "binding": 2,
        "visibility": wgpu.ShaderStage.COMPUTE,
        "buffer": {"type": "storage"}            # pos_out (écriture)
    },
    {
        "binding": 3,
        "visibility": wgpu.ShaderStage.COMPUTE,
        "buffer": {"type": "storage"}            # vel_out (écriture)
    },
    {
        "binding": 4,
        "visibility": wgpu.ShaderStage.COMPUTE,
        "buffer": {"type": "uniform"}            # params (lecture seule)
    },
])

# ------------------------------------------------------------
# Pipeline layout + compute pipeline
# ------------------------------------------------------------
pipeline_layout = device.create_pipeline_layout(bind_group_layouts=[bind_group_layout])

compute_pipeline = device.create_compute_pipeline(
    layout=pipeline_layout,
    compute={"module": shader_module, "entry_point": "main"},
)

# ------------------------------------------------------------
# Bind group : branchement concret des buffers GPU
# ------------------------------------------------------------
bind_group = device.create_bind_group(
    layout=bind_group_layout,
    entries=[
        {"binding": 0, "resource": {"buffer": pos_in,  "offset": 0, "size": positions_np.nbytes}},
        {"binding": 1, "resource": {"buffer": vel_in,  "offset": 0, "size": velocities_np.nbytes}},
        {"binding": 2, "resource": {"buffer": pos_out, "offset": 0, "size": positions_np.nbytes}},
        {"binding": 3, "resource": {"buffer": vel_out, "offset": 0, "size": velocities_np.nbytes}},
        {"binding": 4, "resource": {"buffer": params_buf, "offset": 0, "size": len(params_bytes)}},
    ],
)

print("OK : pipeline Step 2A + bind_group créés")


# dispatch (1 frame)

In [ ]:
# ------------------------------------------------------------
# Dispatch du compute shader
# ------------------------------------------------------------

# Taille d’un workgroup (doit correspondre à @workgroup_size(64) dans le WGSL)
wg_size = 64

# Nombre de workgroups à lancer pour couvrir N particules :
# on fait un "ceil division" pour être sûr de couvrir tous les indices
num_workgroups = (N + wg_size - 1) // wg_size

# Création d’un encodeur de commandes (on enregistre des commandes GPU)
command_encoder = device.create_command_encoder()

# Début d’une passe compute (zone où l’on exécute un compute shader)
compute_pass = command_encoder.begin_compute_pass()

# On sélectionne le pipeline compute (shader + layout)
compute_pass.set_pipeline(compute_pipeline)

# On attache le bind group au slot 0 :
# slot 0 correspond à @group(0) dans le WGSL
#
# Remarque : certaines versions de wgpu acceptent (index, bind_group, dynamic_offsets)
# Ici on fournit une liste vide car on n’utilise pas d’offsets dynamiques.
compute_pass.set_bind_group(0, bind_group, [])

# Lancement du compute shader :
# - X = nombre de workgroups
# - Y, Z = 1 ici car on utilise un index 1D (gid.x)
compute_pass.dispatch_workgroups(num_workgroups, 1, 1)

# Fin de la passe compute
compute_pass.end()

# Soumission des commandes au GPU
queue.submit([command_encoder.finish()])

print("OK : dispatch effectué →", num_workgroups, "workgroups")
print("OK : dispatch effectué →", num_workgroups, "workgroups  (N =", N, ")")



In [ ]:
print("pos_in size bytes:", pos_in.size)
print("positions_np bytes:", positions_np.nbytes)
print("indices bytes:", indices_np.nbytes)
print("first 12 indices:", indices_np[:12])


# “Simulation multi-frames (Step 2A)”

In [ ]:
# # ===============================
# # SIMULATION SUR PLUSIEURS FRAMES
# # ===============================
# # Objectif :
# # - exécuter plusieurs pas de temps
# # - swap pos_in <-> pos_out et vel_in <-> vel_out
# # - observer l'évolution du tissu

# def run_one_frame(pos_in, vel_in, pos_out, vel_out):
#     # Recréation du bind group (simple et sûr)
#     bind_group = device.create_bind_group(
#         layout=bind_group_layout,
#         entries=[
#             {"binding": 0, "resource": {"buffer": pos_in,  "offset": 0, "size": positions_np.nbytes}},
#             {"binding": 1, "resource": {"buffer": vel_in,  "offset": 0, "size": velocities_np.nbytes}},
#             {"binding": 2, "resource": {"buffer": pos_out, "offset": 0, "size": positions_np.nbytes}},
#             {"binding": 3, "resource": {"buffer": vel_out, "offset": 0, "size": velocities_np.nbytes}},
#             {"binding": 4, "resource": {"buffer": params_buf, "offset": 0, "size": len(params_bytes)}},
#         ],
#     )

#     wg_size = 64
#     num_workgroups = (N + wg_size - 1) // wg_size

#     encoder = device.create_command_encoder()
#     compute_pass = encoder.begin_compute_pass()
#     compute_pass.set_pipeline(compute_pipeline)
#     compute_pass.set_bind_group(0, bind_group, [])
#     compute_pass.dispatch_workgroups(num_workgroups, 1, 1)
#     compute_pass.end()

#     queue.submit([encoder.finish()])


# # Nombre de frames à simuler
# num_frames = 60  # ~1 seconde à 60 Hz

# # Reset buffers d'entrée avec les positions initiales CPU
# device.queue.write_buffer(pos_in, 0, positions_np.tobytes())
# device.queue.write_buffer(vel_in, 0, velocities_np.tobytes())


# for frame in range(num_frames):
#     run_one_frame(pos_in, vel_in, pos_out, vel_out)

#     # SWAP : la sortie devient l'entrée pour la frame suivante
#     pos_in, pos_out = pos_out, pos_in
#     vel_in, vel_out = vel_out, vel_in

# print("Simulation multi-frames terminée")


# readback + check

In [ ]:
# # Etat courant = pos_in / vel_in (surtout si tu fais swap)
# raw_p = device.queue.read_buffer(pos_in, 0, positions_np.nbytes)
# positions_after = np.frombuffer(raw_p, dtype=np.float32).reshape((N, 4)).copy()

# # mesure "courbure" au centre: p - moyenne des 4 voisins structuraux
# cx, cy = W//2, H//2
# i = cy*W + cx
# iL = cy*W + (cx-1)
# iR = cy*W + (cx+1)
# iU = (cy-1)*W + cx
# iD = (cy+1)*W + cx

# p  = positions_after[i,:3]
# pm = (positions_after[iL,:3] + positions_after[iR,:3] + positions_after[iU,:3] + positions_after[iD,:3]) / 4.0

# curv = float(np.linalg.norm(p - pm))
# print("\nCourbure centre (norme) =", curv)


# raw_v = device.queue.read_buffer(vel_in, 0, velocities_np.nbytes)
# vel_after = np.frombuffer(raw_v, dtype=np.float32).reshape((N, 4)).copy()

# print("\nEtat courant (pos_in) - 3 points:")
# print(positions_after[:3])

# print("\nEtat courant (vel_in) - 3 points:")
# print(vel_after[:3, :3])

# print("\nvy moyen:", float(np.mean(vel_after[:, 1])))

# print("\nCoin gauche (index 0):", positions_after[0], vel_after[0,:3])
# print("\nCoin droit  (index W-1):", positions_after[W-1], vel_after[W-1,:3])


# mid = (H//2)*W + (W//2)
# print("\nPoint centre index", mid, ":", positions_after[mid], vel_after[mid,:3])


In [ ]:
# C = np.array([0.0, -2.5, 0.0], dtype=np.float32)
# r = 1.0
# dists = np.linalg.norm(positions_after[:, :3] - C[None, :], axis=1)
# print("Nb points à l'intérieur (devrait être 0 après projection) :", int(np.sum(dists < r)))
# print("Nb points proches surface (|dist-r|<0.01):", int(np.sum(np.abs(dists - r) < 0.01)))
# print("min dist-radius:", float(np.min(dists - r)))


# nouvelles cellules qui t'a donné

## setup rendu

In [ ]:
import numpy as np
import wgpu
from wgpu.gui.auto import WgpuCanvas, run

from src.gpu_utils import read_text, create_uniform_buffer

# ---- Canvas / Surface (style Lab3) ----
width_px, height_px = 900, 700
canvas = WgpuCanvas(size=(width_px, height_px))

present_context = canvas.get_context("wgpu")
texture_format = present_context.get_preferred_format(device.adapter)

present_context.configure(
    device=device,
    format=texture_format,
    usage=wgpu.TextureUsage.RENDER_ATTACHMENT,
)

# ---- Matrices (MVP) ----
def perspective(fovy, aspect, znear, zfar):
    f = 1.0 / np.tan(fovy / 2.0)
    M = np.zeros((4, 4), dtype=np.float32)
    M[0, 0] = f / aspect
    M[1, 1] = f
    M[2, 2] = (zfar + znear) / (znear - zfar)
    M[2, 3] = (2 * zfar * znear) / (znear - zfar)
    M[3, 2] = -1.0
    return M

def look_at(eye, target, up):
    eye = np.array(eye, dtype=np.float32)
    target = np.array(target, dtype=np.float32)
    up = np.array(up, dtype=np.float32)

    f = target - eye
    f = f / np.linalg.norm(f)
    s = np.cross(f, up)
    s = s / np.linalg.norm(s)
    u = np.cross(s, f)

    M = np.eye(4, dtype=np.float32)
    M[0, 0:3] = s
    M[1, 0:3] = u
    M[2, 0:3] = -f

    T = np.eye(4, dtype=np.float32)
    T[0, 3] = -eye[0]
    T[1, 3] = -eye[1]
    T[2, 3] = -eye[2]
    return M @ T

aspect = width_px / height_px
P = perspective(np.deg2rad(45.0), aspect, 0.1, 100.0)
V = look_at(eye=(0, 0, 6), target=(0, -1.0, 0), up=(0, 1, 0))
M = np.eye(4, dtype=np.float32)

MVP = (P @ V @ M).astype(np.float32)
cam_buf = create_uniform_buffer(device, MVP.tobytes())

# ---- Shader + bind group ----
shader = device.create_shader_module(code=read_text("shaders/render_basic.wgsl"))

cam_bgl = device.create_bind_group_layout(entries=[
    {"binding": 0, "visibility": wgpu.ShaderStage.VERTEX, "buffer": {"type": "uniform"}},
])

cam_bg = device.create_bind_group(layout=cam_bgl, entries=[
    {"binding": 0, "resource": {"buffer": cam_buf, "offset": 0, "size": MVP.nbytes}},
])

# ---- Pipeline ----
render_pipeline = device.create_render_pipeline(
    layout=device.create_pipeline_layout(bind_group_layouts=[cam_bgl]),
    vertex={
        "module": shader,
        "entry_point": "vs_main",
        "buffers": [{
            "array_stride": 16,      # vec4<f32>
            "step_mode": "vertex",   # <- string (wgpu 0.28.0)
            "attributes": [
                {"shader_location": 0, "offset": 0, "format": "float32x4"},
            ],
        }],
    },
    fragment={
        "module": shader,
        "entry_point": "fs_main",
        "targets": [{"format": texture_format}],
    },
    primitive={"topology": "triangle-list", "cull_mode": "none"},
)

print("OK: canvas + render pipeline ready")


## draw loop

In [ ]:
def draw_frame():
    # Texture écran (swapchain)
    tex = present_context.get_current_texture()
    view = tex.create_view()

    encoder = device.create_command_encoder()

    rp = encoder.begin_render_pass(color_attachments=[{
        "view": view,
        "load_op": "clear",
        "store_op": "store",
        "clear_value": (0.02, 0.02, 0.02, 1.0),
    }])

    rp.set_pipeline(render_pipeline)

    # bind group : PAS d'arguments en plus
    rp.set_bind_group(0, cam_bg, [])

    # vertex + index
    rp.set_vertex_buffer(0, pos_in, 0, positions_np.nbytes)
    rp.set_index_buffer(idx_buf, index_format="uint32")

    # draw
    rp.draw_indexed(indices_np.size, 1, 0, 0, 0)

    rp.end()
    queue.submit([encoder.finish()])

    # animation continue
    canvas.request_draw(draw_frame)

canvas.request_draw(draw_frame)
run()


In [ ]:
# ===== CELLULE DE RENDU =====
import numpy as np
import wgpu
from rendercanvas.jupyter import RenderCanvas  # ← CHANGE ICI
from src.gpu_utils import read_text, create_uniform_buffer

# ---- Canvas pour Jupyter ----
canvas = RenderCanvas(size=(900, 700), max_fps=60)

# ---- Context ----
context = canvas.get_context("wgpu")
texture_format = context.get_preferred_format(device.adapter)
context.configure(device=device, format=texture_format)

# ---- Matrices MVP ----
def perspective(fovy, aspect, znear, zfar):
    f = 1.0 / np.tan(fovy / 2.0)
    M = np.zeros((4, 4), dtype=np.float32)
    M[0, 0] = f / aspect
    M[1, 1] = f
    M[2, 2] = (zfar + znear) / (znear - zfar)
    M[2, 3] = (2 * zfar * znear) / (znear - zfar)
    M[3, 2] = -1.0
    return M

def look_at(eye, target, up):
    eye = np.array(eye, dtype=np.float32)
    target = np.array(target, dtype=np.float32)
    up = np.array(up, dtype=np.float32)
    f = target - eye
    f = f / np.linalg.norm(f)
    s = np.cross(f, up)
    s = s / np.linalg.norm(s)
    u = np.cross(s, f)
    M = np.eye(4, dtype=np.float32)
    M[0, 0:3] = s
    M[1, 0:3] = u
    M[2, 0:3] = -f
    T = np.eye(4, dtype=np.float32)
    T[0, 3] = -eye[0]
    T[1, 3] = -eye[1]
    T[2, 3] = -eye[2]
    return M @ T

aspect = 900 / 700
P = perspective(np.deg2rad(45.0), aspect, 0.1, 100.0)
V = look_at(eye=(0, 0, 6), target=(0, -1.0, 0), up=(0, 1, 0))
M_model = np.eye(4, dtype=np.float32)
MVP = (P @ V @ M_model).astype(np.float32)
cam_buf = create_uniform_buffer(device, MVP.tobytes())

# ---- Shader de rendu ----
shader = device.create_shader_module(code=read_text("shaders/render_basic.wgsl"))

# ---- Bind group caméra ----
cam_bgl = device.create_bind_group_layout(entries=[
    {"binding": 0, "visibility": wgpu.ShaderStage.VERTEX, "buffer": {"type": wgpu.BufferBindingType.uniform}},
])
cam_bg = device.create_bind_group(layout=cam_bgl, entries=[
    {"binding": 0, "resource": {"buffer": cam_buf, "offset": 0, "size": MVP.nbytes}},
])

# ---- Pipeline de rendu ----
render_pipeline = device.create_render_pipeline(
    layout=device.create_pipeline_layout(bind_group_layouts=[cam_bgl]),
    vertex={
        "module": shader,
        "entry_point": "vs_main",
        "buffers": [{
            "array_stride": 16,  # vec4<f32>
            "step_mode": wgpu.VertexStepMode.vertex,
            "attributes": [{"shader_location": 0, "offset": 0, "format": wgpu.VertexFormat.float32x4}],
        }],
    },
    fragment={
        "module": shader,
        "entry_point": "fs_main",
        "targets": [{"format": texture_format}],
    },
    primitive={"topology": wgpu.PrimitiveTopology.triangle_list, "cull_mode": wgpu.CullMode.none},
    depth_stencil=None,
)

print("✅ Render pipeline ready")

# ---- Draw loop ----
@canvas.request_draw
def draw_frame():
    screen_texture = context.get_current_texture()
    encoder = device.create_command_encoder()
    
    rp = encoder.begin_render_pass(color_attachments=[{
        "view": screen_texture.create_view(),
        "load_op": wgpu.LoadOp.clear,
        "store_op": wgpu.StoreOp.store,
        "clear_value": (0.02, 0.02, 0.02, 1.0),
    }])
    
    rp.set_pipeline(render_pipeline)
    rp.set_bind_group(0, cam_bg, [], 0, 999999)
    rp.set_vertex_buffer(0, pos_in, 0, positions_np.nbytes)
    rp.set_index_buffer(idx_buf, wgpu.IndexFormat.uint32, 0, indices_np.nbytes)
    rp.draw_indexed(indices_np.size, 1, 0, 0, 0)
    rp.end()
    
    device.queue.submit([encoder.finish()])

canvas.request_draw(draw_frame)

# Afficher le canvas
canvas
